# QTensorAI Tutorial

To start using QTensorAI for your own hybrid quantum-classical neural network research, you need to do a few things.
1. Write your own circuit composer.
2. Write your own hybrid pytorch neural network module.

The circuit composer is what you use to create your circuit, and the hybrid module is what you use to integrate with a machine learning pipeline.

We have built base classes of circuit composers and hybrid modules that allows you to focus on the science of your study, rather than the implementation details of our library.

## Custom Circuit Composer

Let's say we want to create a straight forward quantum neural network as is described by Abbas et. al. in arXiv:2011.00027. We will not focus on the circuit, but on how you can use our library. First, let us import the class that facilitates your creation of a custom circuit composer. This class is the `ParallelComposer` class.

In [1]:
from qtensor_ai import ParallelComposer

/home/hl8967/.conda/envs/qtensor_ai/lib/python3.9/site-packages/tqdm-4.63.0-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Then, we can create our own composer inheriting from the `ParallelComposer` class. This composer is also provided in `examples/Custom_Circuit_Composers.py`. The comments in this notebook are made with a narrative flow and is best to read without skipping.

In [2]:
class QNNComposer(ParallelComposer):
    
    '''For initialization, n_qubits is always needed since the ParallelComposer class requires that
    The other parameters are specific to this example and are not important.'''
    def __init__(self, n_qubits, n_layers, higher_order=False):
        self.n_layers = n_layers
        self.higher_order = higher_order
        super().__init__(n_qubits)
    
    '''This creates a layer of Hadamard gates. To apply any gates,
    use self.apply_gate(self.operators.gate, *qubits, **parameters).
    There can be mulple qubits or parameters, depending on the gate type.
    Other functions below are logics to add gates. We can ignore them for now.'''
    def layer_of_Hadamards(self):
        for q in self.qubits:
            self.apply_gate(self.operators.H, q)
    
    def entangling_layer(self):
        for i in range(self.n_qubits//2):
            control_qubit = self.qubits[2*i]
            target_qubit = self.qubits[2*i+1]
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)
        for i in range((self.n_qubits+1)//2-1):
            control_qubit = self.qubits[2*i+1]
            target_qubit = self.qubits[2*i+2]
            '''For example, you can put two qubits for CNOT (cX) gates.'''
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)
        control_qubit = self.qubits[-1]
        target_qubit = self.qubits[0]
    
    def encoding_circuit(self, data):
        self.layer_of_Hadamards()
        for i, qubit in enumerate(self.qubits):
            self.apply_gate(self.operators.ZPhase, qubit, alpha=data[:, i])
        if self.higher_order:
            for i in range(self.n_qubits):
                for j in range(i+1, self.n_qubits):
                    control_qubit = self.qubits[i]
                    target_qubit = self.qubits[j]
                    self.apply_gate(self.operators.cX, control_qubit, target_qubit)
                    self.apply_gate(self.operators.ZPhase, target_qubit, alpha=data[:, i]*data[:, j])
                    self.apply_gate(self.operators.cX, control_qubit, target_qubit)
    
    def variational_layer(self, layer, layer_params):
        for i in range(self.n_qubits):
            qubit = self.qubits[i]
            '''For an RY (YPhase) gate, there is one qubit,
            and a alpha parameter which is a torch.Tensor of size (n_batch, 1).'''
            self.apply_gate(self.operators.YPhase, qubit, alpha=layer_params[:, i])
     
    def cost_operator(self):
        for qubit in self.qubits:
            self.apply_gate(self.operators.Z, qubit)

    def forward_circuit(self, data, params):
        self.encoding_circuit(data)
        self.entangling_layer()
        for layer in range(self.n_layers):
            self.variational_layer(layer, params[:, :, layer])
            self.entangling_layer()
    '''The detailed inputs of apply_gate will depend on the gate, and you can implement
    custom gates as well. For those, you could ask for any fancy input parameters.

    Moving on, with all the functions for building the circuit ready, we can implement
    required functions. Specifically, any custom composers must have update_full_circuit
    and name.

    This function builds circuit whose first amplitude is the expectation value of
    the measured circuit w.r.t. the cost_operator.
    This function needs to return the circuit (a list) whose first amplitude you want to simulate.'''
    def updated_full_circuit(self, **parameters):
        data = parameters['data']
        params = parameters['params']
        '''All the apply_gate operations actually appends the gates to self.builder.circuit.
        Hence, after we call the functions that builds the circuits, wee need to fetch the
        circuit from self.builder. We will also need to clean the builder up from time to time.'''
        self.builder.reset() # Clear builder.circuit
        self.forward_circuit(data, params) # Set builder.circuit to the forward circuit according to data and params
        self.cost_operator() # Add the cost operators to builder.circuit
        first_part = self.builder.circuit # Extract builder.circuit at this stage for later use
        self.builder.reset() # Clear builder.circuit
        self.forward_circuit(data, params) # Set builder.circuit to the forward circuit according to data and params
        self.builder.inverse() # Change builder.circuit to it's reverse, which is the forward circuit in reverse
        second_part = self.builder.circuit # Extract the inverse circuit
        self.builder.reset() # Clear builder circuit
        '''The final circuit is forward + cost + inverse.
        The first amplitude is the expectation value of the cost operator
        for the forward circuit initialized with the 0 state.'''
        return first_part + second_part
    '''Although this function returns a circuit, we should not call this function in general.
    This is because the parent class uses this function for the produce_circuit method to
    generate the circuit and tensors on the GPU behind the scene.'''

    '''This function returns the name of the circuit composer'''
    def name(self):
        return 'QNN'

To recap, you need to initialize the `ParallelComposer` class with `n_qubits`, call `self.apply_gate` to add gates to `self.builder.circuit`, and create the `update_full_circuit` method which returns the circuit whose first amplitude you want to simulate. Finally, create the `name` method.

## Custom Hybrid Module

Now, let us move on to creating a custom hybrid module that can interface with a machine learning pipeline. First, import the `HybridModule` class.

In [3]:
from qtensor_ai import HybridModule, DefaultOptimizer
import torch
import torch.nn as nn

We also include the `DefaultOptimizer` class to put in as the default choice for the optimizer. We will create a drop-in replacement of a classical fully connected layer called `QNN`. This can also be found in `examples/Custom_Modules.py`.

In [4]:
'''This is a drop-in replacement of linear layers.
The number of input features is the number of qubits.
Each output feature is computed by an independently parameterized circuit'''
class QNN(HybridModule):
    
    def __init__(self, in_features, out_features, variational_layers=1, higher_order=False, optimizer=DefaultOptimizer()):
                
        '''Initializing module parameters
        The variable circuit_name is needed for initialization of the parent class HybridModule
        All the other attributes are unique to this circuit and we can ignore.'''
        circuit_name = 'n_{}_l_{}'.format(in_features, variational_layers)
        self.higher_order = higher_order
        self.in_features = in_features
        self.out_features = out_features
        self.variational_layers = variational_layers
        self.higher_order = higher_order
        
        '''Define the circuit composer and initialize the hybrid module.
        The composer is the custom composer we just defined.'''
        composer = QNNComposer(in_features, variational_layers, higher_order=higher_order)
        super(QNN, self).__init__(circuit_name=circuit_name, composer=composer, optimizer=optimizer)

        '''self.weight are model weights. Weights must be defined after super().__init__()'''
        self.weight = nn.Parameter(torch.randn(out_features, in_features, variational_layers, dtype=torch.float32))


    def forward(self, x):
        '''These lines of code are trying to manipulate the array to give the right input to the simulation.
        The circuits with different parameters will be simulated in a batch parallel manner.
        The 0-th dimension is the parallel batch dimension.'''
        n_batch = x.shape[0] # (n_batch, in_features)
        # Because for multiple output features, the parallelism is in the batch dimension as well as the
        # output feature size dimension, we need to make the 0-th dimension of size out_features*n_batch
        x = x.repeat(self.out_features, 1) # (out_features*n_batch, in_features)
        params = self.weight.unsqueeze(1) # (out_features, 1, in_features, variational_layers)
        params = params.expand(-1, n_batch, -1, -1) # (out_features, n_batch, in_features, variational_layers)
        params = params.reshape(self.out_features*n_batch, self.in_features, self.variational_layers) # (out_features*n_batch, in_features, variational_layers)

        '''The actual simulation must be run by calling the parent_forward method of the parent class. 
        The parameters should be the same parameters as those accepted by the circuit composer'''
        out = self.parent_forward(data=x, params=params) # (out_features*n_batch)
        # Reshaping the outputs
        out = torch.real(out) # (out_features*n_batch)
        out = out.reshape(self.out_features, n_batch) # (out_features, n_batch)
        out = out.permute(1, 0) # (n_batch, out_features)
        return out

To recap, we need to have a `circuit_name`, initialize the `HybridModule` parent class with your circuit name, composer and optimizer, and only after super init create any model weights. Finally, to run the simulation, `self.parent_forward` must be used in `forward`.

## Bringing Everything Together

Now let us test if our model works.

In [7]:
from qtensor_ai import DefaultOptimizer

in_features = 10
out_features = 10
variational_layers = 1
optimizer = DefaultOptimizer()
epochs = 10

'''Creating an instance of our custom hybrid module'''
qnn = QNN(in_features, out_features, variational_layers=variational_layers, optimizer=optimizer)

'''Make a hybrid neural network'''
model = nn.Sequential(nn.Linear(in_features,in_features),
                    nn.ReLU(),
                    qnn,
                    nn.ReLU(),
                    nn.Linear(out_features,1),
                    nn.ReLU())

'''Define the machine learning optimizer and loss function'''
torch_optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.functional.mse_loss

'''Create some random data. Output is fixed so that we can learn it and see the loss decrease'''
x = torch.rand(1000, in_features)
y = torch.ones(1000, 1)

print('QNN weight before training: ', qnn.weight[1].squeeze())

for epoch in range(epochs):
    y_hat = model(x)
    loss = loss_fn(y, y_hat)
    torch_optim.zero_grad(set_to_none=True)
    loss.backward()
    print(loss)
    torch_optim.step()

print('QNN weight after training: ', qnn.weight[1].squeeze())

QNN weight before training:  tensor([-1.9620,  0.8190,  1.4865,  1.9406, -0.6534,  1.2902, -0.5050,  2.1383,
        -0.0538,  0.1149], grad_fn=<SqueezeBackward0>)
Using previously saved contraction order at  <_io.BufferedReader name='/home/hl8967/Saved_Contraction_Orders/OrderingOptimizer/QNN/n_10_l_1.pickle'>
tensor(0.8355, grad_fn=<MseLossBackward0>)
tensor(0.8332, grad_fn=<MseLossBackward0>)
tensor(0.8309, grad_fn=<MseLossBackward0>)
tensor(0.8286, grad_fn=<MseLossBackward0>)
tensor(0.8263, grad_fn=<MseLossBackward0>)
tensor(0.8239, grad_fn=<MseLossBackward0>)
tensor(0.8216, grad_fn=<MseLossBackward0>)
tensor(0.8191, grad_fn=<MseLossBackward0>)
tensor(0.8167, grad_fn=<MseLossBackward0>)
tensor(0.8142, grad_fn=<MseLossBackward0>)
QNN weight after training:  tensor([-1.9518,  0.8190,  1.4964,  1.9406, -0.6432,  1.2902, -0.4984,  2.1383,
        -0.0538,  0.1251], grad_fn=<SqueezeBackward0>)


We see that the loss indeed decreases and our quantum module parameters also changed after learning.

## Performance and Choosing Optimizers

For simulating larger circuits, one needs to use the `TamakiOptimizer`, which optimizes the contraction order with the state-of-the-art method.

In [14]:
from qtensor_ai import TamakiOptimizer
import time

in_features = 20
out_features = 10
variational_layers = 4
optimizer = TamakiOptimizer(wait_time=20)
epochs = 10

qnn = QNN(in_features, out_features, variational_layers=variational_layers, optimizer=optimizer)

model = nn.Sequential(nn.Linear(in_features, in_features),
                    nn.ReLU(),
                    qnn,
                    nn.ReLU(),
                    nn.Linear(out_features, 1),
                    nn.ReLU())

torch_optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.functional.mse_loss

x = torch.rand(10, in_features)
y = torch.ones(10, 1)

print('QNN weight before training: ', qnn.weight[1,:5].squeeze())

'''The first Contraction Order optimization takes place here, which takes a long time.
To accurately time the computation itself, we only time aftr the first run.'''
y_hat = model(x)
start = time.time()
for epoch in range(epochs):
    y_hat = model(x)
    loss = loss_fn(y, y_hat)
    torch_optim.zero_grad(set_to_none=True)
    loss.backward()
    print(loss)
    torch_optim.step()

stop = time.time()
print('Time taken for training: {} seconds.'.format(stop-start))
print('QNN weight after training: ', qnn.weight[1,:5].squeeze())

QNN weight before training:  tensor([[-0.1719, -0.5676, -0.9996, -0.5885],
        [-1.7975,  0.7700,  0.4302, -1.1895],
        [ 0.1623, -0.5708,  0.0832, -0.4439],
        [ 0.6429,  1.8144,  0.3344,  0.4775],
        [-0.1656,  0.9729, -2.2048,  0.6325]], grad_fn=<SqueezeBackward0>)


Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=None, width=None
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Time=11562, width=11
Exception: Timeout. Stoppnig tamaki


New contraction order, save at  <_io.BufferedWriter name='/home/hl8967/Saved_Contraction_Orders/TamakiOptimizer/QNN/wait_time_20/n_20_l_4.pickle'>
tensor(0.9838, grad_fn=<MseLossBackward0>)
tensor(0.9815, grad_fn=<MseLossBackward0>)
tensor(0.9792, grad_fn=<MseLossBackward0>)
tensor(0.9769, grad_fn=<MseLossBackward0>)
tensor(0.9745, grad_fn=<MseLossBackward0>)
tensor(0.9722, grad_fn=<MseLossBackward0>)
tensor(0.9698, grad_fn=<MseLossBackward0>)
tensor(0.9673, grad_fn=<MseLossBackward0>)
tensor(0.9649, grad_fn=<MseLossBackward0>)
tensor(0.9624, grad_fn=<MseLossBackward0>)
Time taken for training: 3.906189441680908 seconds.
QNN weight after training:  tensor([[-0.1820, -0.5575, -0.9895, -0.5783],
        [-1.7947,  0.7772,  0.4373, -1.1895],
        [ 0.1521, -0.5607,  0.0931, -0.4419],
        [ 0.6530,  1.8245,  0.3296,  0.4775],
        [-0.1581,  0.9791, -2.2118,  0.6426]], grad_fn=<SqueezeBackward0>)


The TamakiOptimizer with 20 seconds of optimization time takes 4 seconds to run 10 epochs. The loss value fluctuates greatly depending on the initial condition and is not indicative of the performance.

In [10]:
in_features = 20
out_features = 10
variational_layers = 4
optimizer = DefaultOptimizer()
epochs = 10

'''Creating an instance of our custom hybrid module'''
qnn = QNN(in_features, out_features, variational_layers=variational_layers, optimizer=optimizer)

'''Make a hybrid neural network'''
model = nn.Sequential(nn.Linear(in_features, in_features),
                    nn.ReLU(),
                    qnn,
                    nn.ReLU(),
                    nn.Linear(out_features, 1),
                    nn.ReLU())

'''Define the machine learning optimizer and loss function'''
torch_optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.functional.mse_loss

'''Create some random data. Output is fixed so that we can learn it and see the loss decrease'''
x = torch.rand(10, in_features)
y = torch.ones(10, 1)

print('QNN weight before training: ', qnn.weight[1,:5].squeeze())

start = time.time()
for epoch in range(epochs):
    y_hat = model(x)
    loss = loss_fn(y, y_hat)
    torch_optim.zero_grad(set_to_none=True)
    loss.backward()
    print(loss)
    torch_optim.step()

stop = time.time()
print('Time taken for training: {} seconds.'.format(stop-start))
print('QNN weight after training: ', qnn.weight[1,:5].squeeze())

QNN weight before training:  tensor([[ 0.3343,  1.4748, -0.2820, -0.6933],
        [ 0.9820, -0.4376,  1.2630, -0.5573],
        [-0.2909,  2.0450,  0.3828,  0.4420],
        [-0.2617,  1.1508,  0.2216, -0.4555],
        [-0.0056,  0.6277, -0.8806,  0.6953]], grad_fn=<SqueezeBackward0>)
New contraction order, save at  <_io.BufferedWriter name='/home/hl8967/Saved_Contraction_Orders/OrderingOptimizer/QNN/n_20_l_4.pickle'>
tensor(0.7491, grad_fn=<MseLossBackward0>)
tensor(0.7472, grad_fn=<MseLossBackward0>)
tensor(0.7454, grad_fn=<MseLossBackward0>)
tensor(0.7436, grad_fn=<MseLossBackward0>)
tensor(0.7418, grad_fn=<MseLossBackward0>)
tensor(0.7400, grad_fn=<MseLossBackward0>)
tensor(0.7382, grad_fn=<MseLossBackward0>)
tensor(0.7364, grad_fn=<MseLossBackward0>)
tensor(0.7346, grad_fn=<MseLossBackward0>)
tensor(0.7328, grad_fn=<MseLossBackward0>)
Time taken for training: 34.151867389678955 seconds.
QNN weight after training:  tensor([[ 3.3190e-01,  1.4721e+00, -2.9022e-01, -6.8696e-01],
   

The default optimizer takes 35 seconds to complete 10 epochs. What is not shown here is the much higher memory consumption as well. In general, using the TamakiOptimizer is highly advantageous for simulating more challenging circuits.